In [1]:
import numpy as np
import pandas as pd
from main import run_di, run_ehd, experiments, save_df
from sim_env import FixedSimEnv
from objects import td
import os
from constants import *

In [6]:
lts = pd.read_csv('data/sim_in/link_times.csv')

In [9]:
trips_by_day = lts.groupby('date')['trip_id'].nunique().reset_index().sort_values(by='trip_id')
worst_days = trips_by_day.iloc[:13]
trips_by_rt_worst = lts[lts['date'].isin(worst_days['date'])].copy()
trips_by_rt_worst = trips_by_rt_worst.groupby(['date', 'route_id'])['trip_id'].nunique().reset_index()

In [10]:
worst_days

,date,trip_id
12,2022-12-01,363
18,2022-12-09,365
10,2022-11-29,368
17,2022-12-08,368
0,2022-11-14,371
11,2022-11-30,376
8,2022-11-25,390
5,2022-11-21,393
7,2022-11-23,393
14,2022-12-05,399


In [11]:
days = ['2022-11-15', '2022-12-08', '2022-11-28']
scenarios = ['NC','EHD','EHD-DI']

pax, trips, events, wlogs, slogs = experiments(dates=days, scenarios=scenarios)

tstamp = pd.Timestamp.today().strftime('%m%d-%H%M')
path_from_cwd = 'data/sim_out/experiments_' + tstamp
os.mkdir(os.path.join(SRC_PATH, path_from_cwd))

save_df(pax, path_from_cwd, 'pax.csv')
save_df(trips, path_from_cwd, 'trips.csv')
save_df(events, path_from_cwd, 'events.csv')
save_df(wlogs, path_from_cwd, 'worklog.csv')
save_df(slogs, path_from_cwd, 'superlog.csv')

while loop:   0%|          | 4/12000 [00:00<05:45, 34.72it/s]

2022-11-15 NC


while loop:   0%|          | 4/12000 [00:00<06:35, 30.31it/s]

2022-11-15 EHD


while loop:   0%|          | 3/12000 [00:00<06:58, 28.66it/s]

...

while loop:   0%|          | 3/12000 [00:00<07:50, 25.48it/s]

2022-11-28 EHD


while loop:   0%|          | 4/12000 [00:00<05:55, 33.70it/s]

2022-11-28 EHD-DI


while loop: 100%|██████████| 12000/12000 [06:56<00:00, 28.79it/s]


In [12]:
pax.head()

,boarding_stop,alighting_stop,arrival_time,times_denied,route_id,boarding_time,trip_id,direction,alighting_time,scenario
0,3737,3741,2022-11-15 05:14:18,0,81,2022-11-15 05:16:39,6279020,East,2022-11-15 05:19:56,NC
1,9561,9565,2022-11-15 05:05:12,0,91,2022-11-15 05:20:53,11565020,South,2022-11-15 05:23:11,NC
2,3744,3745,2022-11-15 05:19:51,0,81,2022-11-15 05:23:02,6279020,East,2022-11-15 05:24:01,NC
3,18106,3746,2022-11-15 05:07:32,0,81,2022-11-15 05:19:21,6279020,East,2022-11-15 05:25:35,NC
4,3741,3746,2022-11-15 05:11:08,0,81,2022-11-15 05:19:56,6279020,East,2022-11-15 05:25:35,NC


In [2]:
env = FixedSimEnv()
unique_dates = env.link_times['date'].unique()
dates = np.random.choice(unique_dates, replace=False, size=3)

In [3]:
next_obs, rew, done, info = env.reset(hist_date='2022-11-21')
print(env.hist_date)

2022-11-21


In [ ]:
env, info, n_steps, ref_veh_df, rts_info = run_di(env, done, debug=True)
print(td(env.time), n_steps)

while loop:   3%|▎         | 367/12000 [00:30<06:55, 27.98it/s]

In [4]:
schd = env.routes['91'].schedule.copy()

In [6]:
schd[(schd['schd_trip_id']==1600020)]['block_id'].iloc[0]

652000002541

In [8]:
dep = schd[(schd['stop_sequence']==1)].iloc[10:23]
dep[dep['stop_headsign']=='Roosevelt'][['block_id', 'schd_trip_id', 'arrival_time', 'direction', 'confirmed']]

,block_id,schd_trip_id,arrival_time,direction,confirmed
480,652000002545,16130020,06:32:00,South,1
582,652000002631,39674020,06:41:30,South,0
781,652000002581,38441020,07:01:00,South,0
880,652000002632,48610020,07:11:00,South,1
1117,652000002541,37199020,07:32:30,South,1
1249,652000002603,4650020,07:43:30,South,0


In [9]:
schd[(schd['block_id']==652000002541) & (schd['stop_sequence']==1)]

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,schd_trip_id,direction,...,monday,tuesday,wednesday,thursday,friday,trip_sequence,runid,departure_time_td,departure_sec,confirmed
138,6520001600020,05:52:30,05:52:30,14103,1,Roosevelt,0,0,1600020,South,...,1,1,1,1,1,4,F169,0 days 05:52:30,21150.0,1
511,6520017504020,06:35:30,06:35:30,9617,1,Jefferson Park Blue Line,0,0,17504020,North,...,1,1,1,1,1,7,F169,0 days 06:35:30,23730.0,1
1117,6520037199020,07:32:30,07:32:30,14103,1,Roosevelt,0,0,37199020,South,...,1,1,1,1,1,13,F169,0 days 07:32:30,27150.0,1
1661,6520019868020,08:21:30,08:21:30,9617,1,Jefferson Park Blue Line,0,0,19868020,North,...,1,1,1,1,1,16,F169,0 days 08:21:30,30090.0,1
2251,6520046669020,09:13:30,09:13:30,14103,1,Roosevelt,0,0,46669020,South,...,1,1,1,1,1,21,F169,0 days 09:13:30,33210.0,1
2696,6520010818020,09:59:30,09:59:30,9617,1,Jefferson Park Blue Line,0,0,10818020,North,...,1,1,1,1,1,24,F169,0 days 09:59:30,35970.0,1
3108,6520032979020,10:48:30,10:48:30,14103,1,Roosevelt,0,0,32979020,South,...,1,1,1,1,1,27,F170,0 days 10:48:30,38910.0,1
3487,6520025207020,11:35:00,11:35:00,9617,1,Jefferson Park Blue Line,0,0,25207020,North,...,1,1,1,1,1,30,F170,0 days 11:35:00,41700.0,1
3883,6520031866020,12:24:30,12:24:30,14103,1,Roosevelt,0,0,31866020,South,...,1,1,1,1,1,33,F170,0 days 12:24:30,44670.0,1
4287,6520004810020,13:13:30,13:13:30,9617,1,Jefferson Park Blue Line,0,0,4810020,North,...,1,1,1,1,1,36,F170,0 days 13:13:30,47610.0,1


In [29]:
env, info, n_steps, ref_veh_df, rts_info, best_switch = run_di(env, done)
print(td(env.time), n_steps)

while loop:  13%|█▎        | 1590/12000 [00:29<03:15, 53.25it/s]

92 81
34110.0 33690.0
92 81
34110.0 33690.0
0.7063422983699943 0.6980647275989049
1.0118578842961472
0 days 07:42:30 1590


In [12]:
from interlining import *

In [29]:
env, info, n_steps, upd_info = run_ehd(env, done, n_steps=n_steps, debug=True)
n_steps

----
Time 0 days 05:55:48
Expected: 0 days 05:56:30
Headways: 0 days 00:08:32 -- 0 days 00:09:00
Schedule: 0 days 05:56:30
0 days 05:55:48 < Departure < 0 days 06:01:30
Recommended 0 days 05:56:44 ---> 0 days 05:56:44


816